In [ ]:

import subprocess
import os
import tempfile

In [ ]:
from pathlib import Path


def merge_mp4_with_ffmpeg(input_files, output_file):
    """
    Merge MP4 files using FFmpeg directly.
    Requires FFmpeg to be installed on the system.
    
    Args:
        input_files (list): List of paths to input MP4 files
        output_file (str): Path for the output merged MP4 file
    """
    try:
        # Create a temporary file list for FFmpeg
        with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
            for file in input_files:
                if os.path.exists(file):
                    # Escape single quotes and write to file list
                    escaped_file = file.replace("'", "'\"'\"'")
                    f.write(f"file '{escaped_file}'\n")
                else:
                    print(f"Warning: File not found: {file}")
            
            filelist_path = f.name
        
        # FFmpeg command to concatenate videos
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', filelist_path,
            '-c', 'copy',  # Copy streams without re-encoding (faster)
            '-y',  # Overwrite output file if it exists
            output_file
        ]
        
        print("Merging videos with FFmpeg...")
        print(f"Command: {' '.join(cmd)}")
        
        # Run FFmpeg
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Clean up temporary file
        os.unlink(filelist_path)
        
        if result.returncode == 0:
            print(f"Successfully merged videos into {output_file}")
            return True
        else:
            print(f"FFmpeg error: {result.stderr}")
            return False
            
    except FileNotFoundError:
        print("Error: FFmpeg not found. Please install FFmpeg first.")
        print("Download from: https://ffmpeg.org/download.html")
        return False
    except Exception as e:
        print(f"Error during merging: {str(e)}")
        return False

def check_ffmpeg():
    """Check if FFmpeg is available on the system."""
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False

# List your MP4 files in the order you want them merged
video_folder =  Path("/home/daniel/d123_logs_videos/wopd_train")
video_files = [str(file) for file in video_folder.glob("*.mp4") if file.is_file()]


output_filename = str(video_folder / "merged_video.mp4")

# Merge the videos
success = merge_mp4_with_ffmpeg(video_files, output_filename)

if success:
    print("Video merging completed successfully!")
else:
    print("Video merging failed!")